<a href="https://colab.research.google.com/github/Kristin33/Composer-Clustering/blob/master/DEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras scikit-learn   

In [0]:
# %cd ..
# %rm -rf my-DEC/

# # 因为我fork在了我的private repo里面，所以还要输个我的密码
# import os
# from getpass import getpass
# user = getpass('GitHub user')
# password = getpass('GitHub password')
# os.environ['GITHUB_AUTH'] = user + ':' + password
# !git clone https://$GITHUB_AUTH@github.com/Kristin33/my-DEC.git
# # %load https://$GITHUB_AUTH@github.com/Kristin33/my-DEC/blob/master/datasets.py

# Load data

In [0]:
# load the compositions dataset
# the piano rolls are pre saved in the directory
# 1280000太大了，出resource exhausted error
def load_comps_pianoroll(files):

    print(files)

    import os
    from PIL import Image
    data_dir = "drive/My Drive/10701/pianoroll/"

    composer_label = np.empty((0,))

    x, y = np.empty((0, 128, 1000)), np.empty((0,))
    for idx, comp in enumerate(files):
        composer_data = np.empty((0, 128, 1000))
        path = data_dir + comp
        for filename in os.listdir(path):
            img = np.asarray(Image.open("{}/{}".format(path, filename)).convert("L"))
            try:
                composer_data = np.append(composer_data, np.expand_dims(img, axis=0), axis=0)

            except:
              pass
        x = np.append(x, composer_data, axis=0)
        y = np.append(y, np.ones((composer_data.shape[0],)) * idx, axis=0)

    # standarize?
    x = stats.zscore(x)
    x = np.nan_to_num(x)

    # x = np.vstack((data1, data2))
    # y = np.hstack((np.zeros(data1.shape[0]), np.ones(data2.shape[0])))

    x = x.reshape((x.shape[0], -1))
    assert x.shape[0] == y.shape[0]

    return x, y


# load the compositions dataset
# the csvs are pre saved in the directory
# csv dimension 776
def load_comps_csv(expr):
    csv_comps = ["dandrieu_all.csv", "dvorak_all.csv", "debussy_all.csv", "handel_all.csv"]

    alkan_csv = "alkan_all.csv"
    bach_csv = "bach_all.csv"
    beethoven_csv = "beethoven_all.csv"
    brahms_csv = "brahms_all.csv"
    buxtehude_csv = "buxtehude_all.csv"
    byrd_csv = "byrd_all.csv"
    chopin_csv = "chopin_all.csv"
    dandrieu_csv = "dandrieu_all.csv"
    dvorak_csv = "dvorak_all.csv"
    debussy_csv = "debussy_all.csv"
    faure_csv = "faure_all.csv"
    handel_csv = "handel_all.csv"
    haydn_csv = "haydn_all.csv"
    mozart_csv = "mozart_all.csv"
    scarlatti_csv = "scarlatti_all.csv"
    schubert_csv = "schubert_all.csv"
    schumann_csv = "schumann_all.csv"
    scriabin_csv = "scriabin_all.csv"
    shostakovich_csv = "shostakovich_all.csv"
    soler_csv = "soler_all.csv"

    f1_e1 = [dandrieu_csv, soler_csv]
    f1_e2 = [dvorak_csv, schumann_csv]
    f1_e3 = [buxtehude_csv, faure_csv]
    f1_e4 = [scriabin_csv, byrd_csv]
    f1_e5 = [shostakovich_csv, brahms_csv]
    f1_e6 = [chopin_csv, debussy_csv]
    f1_e7 = [schubert_csv, alkan_csv]
    f1_e8 = [handel_csv, mozart_csv]
    f1_e9 = [haydn_csv, beethoven_csv]
    f1_e10 = [scarlatti_csv, bach_csv]

    expr_mappings = {
        "f1_e1": f1_e1,
        "f1_e2": f1_e2,
        "f1_e3": f1_e3,
        "f1_e4": f1_e4,
        "f1_e5": f1_e5,
        "f1_e6": f1_e6,
        "f1_e7": f1_e7,
        "f1_e8": f1_e8,
        "f1_e9": f1_e9,
        "f1_e10": f1_e10,
    }


    data_dir = "../drive/My Drive/10701/composer_csv/"

    composer_data = np.empty((0, 776))
    composer_label = np.empty((0,))

    for idx, filename in enumerate(expr_mappings[expr]):
        data = np.genfromtxt(data_dir + filename, delimiter=',', 
                             usecols=np.arange(1,777), encoding="latin1")
        # get rid of the top row with feature names
        data = data[1:,:]
        # standarize the data
        data = stats.zscore(data)
        data = np.nan_to_num(data)

        composer_data = np.append(composer_data, data, axis=0)
        composer_label = np.append(composer_label, np.ones((data.shape[0],)) * idx, axis=0)

    x = composer_data
    y = composer_label

    assert x.shape[0] == y.shape[0]

    # 所有的data只需要一个x和y, x是data，y是label
    # x就是一个numpy matrix (n, dim), y就是(n, )
    return x, y


In [0]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from sklearn.metrics import v_measure_score, adjusted_rand_score

vms = v_measure_score
nmi = normalized_mutual_info_score
ari = adjusted_rand_score


def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed

    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

# DEC code

In [0]:
"""
Keras implementation for Deep Embedded Clustering (DEC) algorithm:

        Junyuan Xie, Ross Girshick, and Ali Farhadi. Unsupervised deep embedding for clustering analysis. ICML 2016.

Usage:
    use `python DEC.py -h` for help.

Author:
    Xifeng Guo. 2017.1.30
"""

from time import time
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
import sklearn
from scipy import stats
import os


def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    x = Input(shape=(dims[0],), name='input')
    h = x

    # internal layers in encoder
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(h)

    # hidden layer
    h = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(h)  # hidden layer, features are extracted from here

    y = h
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        y = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(y)

    # output
    y = Dense(dims[0], kernel_initializer=init, name='decoder_0')(y)

    return Model(inputs=x, outputs=y, name='AE'), Model(inputs=x, outputs=h, name='encoder')


class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class DEC(object):
    def __init__(self,
                 dims,
                 n_clusters=10,
                 alpha=1.0,
                 init='glorot_uniform'):

        super(DEC, self).__init__()

        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.autoencoder, self.encoder = autoencoder(self.dims, init=init)

        # prepare DEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(self.encoder.output)
        self.model = Model(inputs=self.encoder.input, outputs=clustering_layer)

    def pretrain(self, x, y=None, optimizer='adam', epochs=200, batch_size=256, save_dir='results/temp'):
        print('...Pretraining...')
        self.autoencoder.compile(optimizer=optimizer, loss='mse')

        # csv_logger = callbacks.CSVLogger(save_dir + '/pretrain_log.csv')
        # cb = [csv_logger]
        if y is not None:
            class PrintACC(callbacks.Callback):
                def __init__(self, x, y):
                    self.x = x
                    self.y = y
                    super(PrintACC, self).__init__()

                def on_epoch_end(self, epoch, logs=None):
                    if int(epochs/10) != 0 and epoch % int(epochs/10) != 0:
                        return
                    feature_model = Model(self.model.input,
                                          self.model.get_layer(
                                              'encoder_%d' % (int(len(self.model.layers) / 2) - 1)).output)
                    features = feature_model.predict(self.x)
                    km = KMeans(n_clusters=len(np.unique(self.y)), n_init=20, n_jobs=4)
                    y_pred = km.fit_predict(features)
                    # print()
                    print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                          % (acc(self.y, y_pred), nmi(self.y, y_pred)))

            PrintACC(x, y)

        # begin pretraining
        t0 = time()
        self.autoencoder.fit(x, x, batch_size=batch_size, epochs=epochs)
        print('Pretraining time: %ds' % round(time() - t0))
        # self.autoencoder.save_weights(save_dir + '/ae_weights.h5')
        print('Pretrained weights are saved to %s/ae_weights.h5' % save_dir)
        self.pretrained = True

    def load_weights(self, weights):  # load weights of DEC model
        self.model.load_weights(weights)

    def extract_features(self, x):
        return self.encoder.predict(x)

    def predict(self, x):  # predict cluster labels using the output of clustering layer
        q = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def compile(self, optimizer='sgd', loss='kld'):
        self.model.compile(optimizer=optimizer, loss=loss)

    def fit(self, x, y=None, maxiter=2e4, batch_size=256, tol=1e-3,
            update_interval=140, save_dir='./results/temp'):

        print('Update interval', update_interval)
        save_interval = max(int(x.shape[0] / batch_size) * 5, 5)  # 5 epochs
        print('Save interval', save_interval)

        # Step 1: initialize cluster centers using k-means
        t1 = time()
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # Step 2: deep clustering
        # logging file
        import csv
        # logfile = open(save_dir + '/dec_log.csv', 'w')
        # logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'loss'])
        # logwriter.writeheader()

        loss = 0
        index = 0
        index_array = np.arange(x.shape[0])
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q = self.model.predict(x, verbose=0)
                # print(q.shape)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                if y is not None:
                    acc_ = np.round(acc(y, y_pred), 5)
                    nmi_ = np.round(nmi(y, y_pred), 5)
                    ari_ = np.round(ari(y, y_pred), 5)
                    vm = np.round(sklearn.metrics.v_measure_score(y, y_pred), 5)
                    loss = np.round(loss, 5)
                    print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f, vm = %.5f' % (ite, acc_, nmi_, ari_, vm), ' ; loss=', loss)

                # check stop criterion
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    # logfile.close()
                    break

            # train on batch
            # if index == 0:
            #     np.random.shuffle(index_array)
            idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
            loss = self.model.train_on_batch(x=x[idx], y=p[idx])
            index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

            # save intermediate model
            if ite % save_interval == 0:
                print('saving model to:', save_dir + '/DEC_model_' + str(ite) + '.h5')
                # self.model.save_weights(save_dir + '/DEC_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        # logfile.close()
        print('saving model to:', save_dir + '/DEC_model_final.h5')
        # self.model.save_weights(save_dir + '/DEC_model_final.h5')

        return y_pred


def DEC_main(dataset, files, save=False):
    # # setting the hyper parameters
    # import argparse

    # parser = argparse.ArgumentParser(description='train',
    #                                  formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('--dataset', default='mnist',
    #                     choices=['mnist', 'fmnist', 'usps', 'reuters10k', 'stl', 'comps', 'comps_csv'])
    # parser.add_argument('--experiment', default='f1_e1',
    #                     choices=['f1_e1', 'f1_e2', 'f1_e3', 'f1_e4', 'f1_e5', 'f1_e6', 
    #                     'f1_e7', 'f1_e8', 'f1_e9', 'f1_e10'])
    # parser.add_argument('--batch_size', default=256, type=int)
    # parser.add_argument('--maxiter', default=2e4, type=int)
    # parser.add_argument('--pretrain_epochs', default=None, type=int)
    # parser.add_argument('--update_interval', default=None, type=int)
    # parser.add_argument('--tol', default=0.001, type=float)
    # parser.add_argument('--ae_weights', default=None)
    # parser.add_argument('--save_dir', default='results')
    # args = parser.parse_args()
    # print(args)

    # load dataset
    print("loading dataset...")
    x, y = load_comps_pianoroll(files)
    n_clusters = len(np.unique(y))

    print(x.shape)
    print(y.shape)

    init = 'glorot_uniform'
    pretrain_optimizer = 'adam'
    # setting parameters
    if dataset == 'comps_csv':
        update_interval = 140
        pretrain_epochs = 300
        init = VarianceScaling(scale=1. / 3., mode='fan_in',
                               distribution='uniform')  # [-limit, limit], limit=sqrt(1./fan_in)
        pretrain_optimizer = SGD(lr=1, momentum=0.9)
    elif dataset == 'comps':
        update_interval = 30
        pretrain_epochs = 100

    # prepare the DEC model
    dec = DEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=n_clusters, init=init)

    dec.pretrain(x=x, y=y, optimizer=pretrain_optimizer,
                  epochs=pretrain_epochs)

    dec.model.summary()
    t0 = time()
    dec.compile(optimizer=SGD(0.01, 0.9), loss='kld')
    y_pred = dec.fit(x, y=y, update_interval=update_interval)
    print('acc:', acc(y, y_pred))
    print('clustering time: ', (time() - t0))

    if save:
      name = "_".join([f[:5] for f in files]) 
      drive_dir = "drive/My Drive/10701/"
      with open(drive_dir + "results/dec_pr/f3/{}.txt".format(name), "w") as f:
        f.write(", ".join([f[:-4] for f in files]) + "\n")
        f.write("acc: {}\n".format(acc(y, y_pred)))
        f.write("vms: {}\n".format(vms(y, y_pred)))
        f.write("nmi: {}\n".format(nmi(y, y_pred)))
        f.write("ari: {}\n".format(ari(y, y_pred)))
   

    return acc(y, y_pred)


In [0]:
# %cd my-DEC
# %load datasets.py
# from datasets import load_comps_csv

In [0]:
# x, y = load_comps_csv()

In [0]:
# %cd my-DEC/
# !python DEC.py --dataset comps --experiment f1_e5

In [0]:
%cd content/

[Errno 2] No such file or directory: 'content/'
/content


# run all

In [0]:
# 直接的midi数据
alkan_dir = "alkan_(c)contributors-kunstderfuge/"
bach_dir = "bach_all/"
dandrieu_dir = "dandrieu_(c)contributors-kunstderfuge/"
dvorak_dir = "dvorak_(c)contributors-kunstderfuge/"
scriabin_dir = "scriabin_(c)contributors-kunstderfuge/"
byrd_dir = "byrd_(c)contributors-kunstderfuge/"
faure_dir = "faure_(c)contributors-kunstderfuge/"
buxtehude_dir = "buxtehude_(c)contributors-kunstderfuge/"
beethoven_dir = "beethoven_all/"
brahms_dir = "brahms_all/"
schubert_dir = "schubert_all/"
chopin_dir = "chopin_all/"
debussy_dir = "debussy_all/"
handel_dir = "handel_all/"
haydn_dir = "haydn_all/"
scarlatti_dir = "scarlatti_all/"
mozart_dir = "mozart_all/"
schumann_dir = "schumann_(c)contributors-kunstderfuge/"
scriabin_dir = "scriabin_(c)contributors-kunstderfuge/"
shostakovich_dir = "shostakovich_(c)contributors-kunstderfuge/"
soler_dir = "soler_(c)contributors-kunstderfuge/"

pr_comps = [alkan_dir, bach_dir, beethoven_dir,
               brahms_dir, buxtehude_dir, byrd_dir, 
               chopin_dir, dandrieu_dir, dvorak_dir, debussy_dir,
               faure_dir, handel_dir, haydn_dir,
               mozart_dir, scarlatti_dir, schubert_dir,
              schumann_dir, scriabin_dir, shostakovich_dir, soler_dir]

# all_pairs = [[a, b] for a in csv_comps for b in csv_comps if (a != b and a > b)]
# for p in list(all_pairs):
# 	print(p)
# 	run_clustering("kmeans", p)
# 	print("done!")
# data = []
# for a in pr_comps:
#   tmp = []
#   for b in pr_comps:
#     name = "_".join([f[:5] for f in [a, b]]) 
#     if not os.path.exists("drive/My Drive/10701/results/dec_pr/f1/{}.txt".format(name)):
#       tmp.append(DEC_main("comps", [a, b], save=True))
#   data.append(tmp)
# data = np.array(data)

f2_e1 = [scarlatti_dir, mozart_dir, faure_dir, shostakovich_dir]
f2_e2 = [byrd_dir, handel_dir, chopin_dir, scriabin_dir]
f2_e3 = [bach_dir, beethoven_dir, schubert_dir, debussy_dir]
f2_e4 = [buxtehude_dir, haydn_dir, brahms_dir, alkan_dir]

f3_e1 = [scarlatti_dir, bach_dir, byrd_dir, buxtehude_dir]
f3_e2 = [beethoven_dir, handel_dir, mozart_dir, haydn_dir]
f3_e3 = [brahms_dir, chopin_dir, schubert_dir, faure_dir]
f3_e4 = [scriabin_dir, shostakovich_dir, debussy_dir, alkan_dir]

DEC_main("comps", f3_e1, save=True)
DEC_main("comps", f3_e2, save=True)
DEC_main("comps", f3_e3, save=True)
DEC_main("comps", f3_e4, save=True)

import sys
sys.exit()

# print(run_clustering_features("kmeans", [dandrieu_csv, soler_csv], save=True))
np.savetxt("drive/My Drive/10701/results/dec_pr_accuracy.txt", data)
data = np.around(np.loadtxt("drive/My Drive/10701/results/dec_pr_accuracy.txt.txt"), decimals=2)

print(data)
# sys.exit()

fig, ax = plt.subplots()
im = ax.imshow(data, cmap="Wistia")

comps_names = [c[:-8] for c in csv_comps]

# We want to show all ticks...
ax.set_xticks(np.arange(len(comps_names)))
ax.set_yticks(np.arange(len(comps_names)))
# ... and label them with the respective list entries
ax.set_xticklabels(comps_names)
ax.set_yticklabels(comps_names)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(comps_names)):
    for j in range(len(comps_names)):
        text = ax.text(j, i, data[i, j],
                        ha="center", va="center", color="black", size="6")

ax.set_title("heatmap of pairwise composer clustering acc - raw piano roll")
fig.tight_layout()
plt.show()

# 

loading dataset...
['scarlatti_all/', 'bach_all/', 'byrd_(c)contributors-kunstderfuge/', 'buxtehude_(c)contributors-kunstderfuge/']


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


(1741, 128000)
(1741,)
...Pretraining...
Epoch 1/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5870
Epoch 2/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5869
Epoch 3/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5887
Epoch 4/100
1741/1741 [==============================] - 3s 1ms/step - loss: 0.5853
Epoch 5/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5821
Epoch 6/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5812
Epoch 7/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5859
Epoch 8/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5844
Epoch 9/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5922
Epoch 10/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5829
Epoch 11/100
1741/1741 [==============================] - 3s 2ms/step - loss: 0.5765
Epoch 12/100
1741/1741 [=========

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Iter 0: acc = 0.64101, nmi = 0.10949, ari = 0.16592, vm = 0.10949  ; loss= 0
saving model to: ./results/temp/DEC_model_0.h5


...

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Iter 570: acc = 0.32155, nmi = 0.01871, ari = 0.01396, vm = 0.01871  ; loss= 0.01539
saving model to: ./results/temp/DEC_model_570.h5
saving model to: ./results/temp/DEC_model_580.h5
saving model to: ./results/temp/DEC_model_590.h5
Iter 600: acc = 0.32155, nmi = 0.01871, ari = 0.01396, vm = 0.01871  ; loss= 0.01341
delta_label  0.0 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/DEC_model_final.h5
acc: 0.3215547703180212
clustering time:  91.62260103225708


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# 